## Figures

This notebook allows to display the training of a run.

In [ ]:
import os
import math
import pickle

import yaml
import numpy as np

import mtrnn
from mtrnn import graphs

In [ ]:
colors = ["#405f80", "#851d46", "#be2e3A", "#e0623d", "#ebc54b", "grey"]
color_learn, color_fixed = '#ff3d7f', '#3fb8Af'

## Loading Computation

In [ ]:
def load_run(cfg_name):
    cwd = os.getcwd()
    filepath = './exps/{}.yaml'.format(cfg_name)
    run = mtrnn.Run.from_configfile(filepath, resume=True, verbose=False)
    run.load_logs()
    os.chdir(cwd)
    return run

In [ ]:
results = {}
for name in ['cst8', 'cst100', 'invexp', 'exp']:
    run_fixed    = load_run('fixed_{}'.format(name)) if name != 'exp' else None
    run_learntau = load_run('learntau_{}'.format(name))
    results[name] = (run_fixed, run_learntau)

## Plotting Functions

### τ Evolution

In [ ]:
def fig_τ_evolution(name):
    """Figure for showing how τ evolves across epochs"""
    run = results[name][1]
    data = {}

    max_τ = 0
    for frame in run.logs.logs_for('τ'):
        layer_index, τ = frame[1]
        max_τ = max(max_τ, float(τ))
        data.setdefault(layer_index, [])
        data[layer_index].append(τ)

    figure = graphs.Figure(x_range=None, y_range=[0, max_τ+1], title='{} - τ evolution'.format(name))
    for layer_i, τs in data.items():
        figure.line(range(len(τs))[::10], τs[::10], color=colors[layer_i], 
                    alpha=0.5, line_width=2, label=str(layer_i), name='layer {}'.format(layer_i))
    figure.show()
    #graphs.save_fig(figure.fig, 'tau_{}'.format(exp_name), 'png')

### Loss

In [ ]:
def fig_loss(name):
    run_fixed, run_learn = results[name]
    epochs_learn = [frame[2][0]            for frame in run_learn.logs.logs_for('loss')]
    loss_learn   = [float(frame[1]['L_x']) for frame in run_learn.logs.logs_for('loss')]
    epochs_fixed = [frame[2][0]            for frame in run_fixed.logs.logs_for('loss')]
    loss_fixed   = [float(frame[1]['L_x']) for frame in run_fixed.logs.logs_for('loss')]

    title = '{} - loss (MSE from target)'.format(name)
    figure = graphs.Figure(x_range=None, y_range=None, title=title)
    figure.circle(epochs_learn, loss_learn, color=color_learn, alpha=0.25, size=1.0)
    figure.circle(epochs_fixed, loss_fixed, color=color_fixed, alpha=0.25, size=1.0)
    figure.show()

### Closed-Loop Error

In [ ]:
def fig_closed_loop_error(name):
    run_fixed, run_learn = results[name]
    epochs_learn = [frame[2][0] for frame in run_learn.logs.logs_for('error_CL_train')]
    ecl_learn    = [frame[1]    for frame in run_learn.logs.logs_for('error_CL_train')]
    epochs_fixed = [frame[2][0] for frame in run_fixed.logs.logs_for('error_CL_train')]
    ecl_fixed    = [frame[1]    for frame in run_fixed.logs.logs_for('error_CL_train')]

    figure = graphs.Figure(x_range=None, y_range=[0, 60], title='closed-loop error (MSE)')
    figure.circle(epochs_fixed, ecl_fixed, color=color_fixed, alpha=0.25, size=1.0)
    figure.circle(epochs_learn, ecl_learn, color=color_learn, alpha=0.25, size=1.0)
    figure.show()

### Closed and Open Loop Output

In [ ]:
def fig_output(name, sample_index, loop_type='open loop', y_range=(-2, 2)):
    run_fixed, run_learn = results[name]
    dataset = run_fixed.train_dataset
    
    x = range(99) # np.arange(0.01, 1, 0.01)
    y_target = [e[0].item() for e in dataset.sample(sample_index)[1]]

    title = '{} - {}, sample #{}'.format(name, loop_type, sample_index)
    fig = graphs.OutputFigure(x_range=None, y_range=y_range, title=title)    
    fig.line(x, y_target, name='target', color='grey')

    if loop_type == 'open loop':
        x = x[1:]
        y_fixed = [e[0].item() for e in run_fixed.task.open_loop(dataset.batch([sample_index]))][:-1]
        y_learn = [e[0].item() for e in run_learn.task.open_loop(dataset.batch([sample_index]))][:-1]
    else:
        y_fixed = [e[0].item() for e in run_fixed.task.closed_loop(dataset.batch([sample_index]))]
        y_learn = [e[0].item() for e in run_learn.task.closed_loop(dataset.batch([sample_index]))]
    fig.line(x, y_fixed, name='fixed', color=color_fixed, line_width=2, alpha=0.50)
    fig.line(x, y_learn, name='learn', color=color_learn, line_width=2, alpha=0.50)

    fig.show()

## Figure 1

In [ ]:
fig_τ_evolution('cst100')
fig_output('cst100', 0, loop_type='open loop')
fig_output('cst100', 0, loop_type='closed loop')

## Figure 2

In [ ]:
fig_τ_evolution('invexp')
fig_loss('invexp')
fig_output('invexp', 3, loop_type='closed loop', y_range=(-4, 4))

## Figure 3

In [ ]:
fig_τ_evolution('exp')
fig_τ_evolution('cst8')

## Figure 4

In [ ]:
fig_output('cst8', 1, loop_type='open loop')
fig_output('cst8', 1, loop_type='closed loop')
fig_closed_loop_error('cst8')